In [2]:
import numpy as np
from urllib.parse import quote
from flatten_json import flatten
import urllib.request
from bs4 import BeautifulSoup
import requests
import time, re, json, copy, os, random
import shutil
from pprint import pprint
from collections import defaultdict, Counter

In [24]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [3]:
USER_AGENT_LIST = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
            "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
            "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
            "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
        ]

In [4]:
with open("/home/yingshac/CYS/WebQnA/WebQnA_data/img_metadata-Copy1.json", "r") as f:
    img_meta = json.load(f)
print(len(img_meta))

362367


In [5]:
import unicodedata
import string
TABLE = str.maketrans(dict.fromkeys(string.punctuation)) 
def simplify(text):
    
    try:
        text = unicode(text, 'utf-8')
    except NameError:
        pass
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8").strip()
    return str(text)

In [6]:
def get_description_by_url(desurl):
    req = urllib.request.Request(desurl, headers = {'User-Agent': random.choice(USER_AGENT_LIST)})
    try:
        with urllib.request.urlopen(req) as f:
            html = f.read().decode('utf-8')
    except KeyboardInterrupt: raise
    except:
        return "NOTFOUND"

    soup = BeautifulSoup(html, 'html.parser')
    des = ""
    result = soup.find_all('div', class_="description mw-content-ltr en")
    for r in result:
        row = r.text.strip()
        if row[:8] == 'English:':
            row = row[8:]
        des += simplify(row) + " "
    des = des.strip()
        
    if len(des) == 0:
        result = soup.find_all('td',class_= "description")
        
        for r in result:
            row = r.text.strip()
            if row[:8] == 'English:':
                row = row[8:]
            des += simplify(row) + " "
        des = des.strip()
    return des

In [ ]:
result = {}
count = 0
before = 0
after = 0
for k in sorted(list(img_meta.keys()))[0:500]:
    result[k] = copy.deepcopy(img_meta[k])
    count += 1
    if len(img_meta[k]['description'].strip()) == 0:
        before += 1
        x = get_description_by_url(img_meta[k]['page'])
        if x == "NOTFOUND" or len(x.translate(TABLE).strip())==0: 
            x = ""
        result[k]['description'] = x
        if len(x) == 0: after += 1

    if count % 100 == 0:
        print(count, before, after)
        before = 0
        after = 0
        #json.dump(result, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_meta_upd/img_meta_{}.json".format(args.boundary), "w"), indent=4)

100 82 2
200 85 20
300 89 2
400 89 5
500 87 6
600 90 6
700 95 7
800 90 9
900 89 4
1000 94 5
1100 82 6
1200 86 12
1300 90 3
1400 91 8
1500 86 5
1600 84 6
1700 91 14
1800 87 13
1900 84 5
2000 88 6
2100 87 1
2200 85 6
2300 89 7
2400 84 3
2500 88 4
2600 89 10
2700 89 6
2800 86 11
2900 84 3
3000 81 3
3100 90 3
3200 90 5
3300 88 3
3400 92 2
3500 83 12


In [115]:
get_description_by_url("https://commons.wikimedia.org/wiki/File:Rock_Choir_at_Easton_Lodge_Gardens_open_day%2C_Little_Easton%2C_Essex%2C_England_01.jpg")

'The Rock Choir on the croquet lawn at Easton Lodge Gardens in Little Easton, Essex, England, on 20 May 2018.Camera: Canon EOS 6D Mark II with Canon EF 24-105mm F4L IS USM lens.Software: File lens-corrected, optimized, perhaps cropped, with DxO PhotoLab, and likely further optimized with Adobe Photoshop CS2.'

In [37]:
### check progress
path = "/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_meta_upd_shorter/"
#print(os.listdir(path))
img_meta_upd = {}
if os.path.isdir(path):
    finished_samples = []
    files = os.listdir(path)
    print("{} files found".format(len(files)))
    for f in files:
        if not '.json' in f: continue
        try: img_meta_upd.update(json.load(open(os.path.join(path, f), "r")))
        except: raise
else:
    img_meta_upd.update(json.load(open(path, "r")))
for k in img_meta_upd:
    if len(str(k)) < 8:
        imgid = "0"*(8-len(str(k))) + str(k)
        img_meta_upd[imgid] = copy.deepcopy(img_meta_upd[k])
        del img_meta_upd[k]

print("{} samples found".format(len(img_meta_upd)))
description_lengths = [len(img_meta_upd[k]['description']) for k in img_meta_upd if len(img_meta_upd[k]['description'])<10]
print(len(description_lengths), Counter(description_lengths))

19 files found
362367 samples found
188127 Counter({0: 186692, 9: 314, 8: 269, 6: 264, 7: 260, 5: 144, 3: 131, 4: 35, 1: 16, 2: 2})


In [54]:
for k in img_meta_upd:
    des = copy.deepcopy(img_meta_upd[k]['description'])
    if len(des) > 300:
        if "Identifier: " in des: img_meta_upd[k]['description'] = ""
        else:
            D = nlp(des)
            S = [s for s in D.sents]
            if len(str(S[0])) > 300: img_meta_upd[k]['description'] = str(S[0]).split("\n")[0]
            else: img_meta_upd[k]['description'] = str(S[0])
print("{} samples found".format(len(img_meta_upd)))
description_lengths = [len(img_meta_upd[k]['description']) for k in img_meta_upd if len(img_meta_upd[k]['description'])<10]
print(len(description_lengths), Counter(description_lengths))

362367 samples found
189813 Counter({0: 187438, 1: 818, 9: 343, 8: 282, 7: 270, 6: 267, 5: 166, 3: 137, 4: 86, 2: 6})


In [55]:
# re-mined descriptions for wikimedia-commons images
# changed keys to str len=8 format
json.dump(img_meta_upd, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_meta_0901_shorter_truncate.json", "w"), indent=4)

In [ ]:
count = 0
for k in img_meta_upd:
    if len(img_meta_upd[k]['description']) > 300:
        count += 1
        print(k)
        pprint(img_meta_upd[k])
print(count)

10200071
{'description': 'U.S. Navy Engineman 1st Class Don Barron, right, blows out '
                'the candles on his birthday cake given to him as a surprise '
                'by the engineering department aboard the littoral combat ship '
                'USS Freedom (LCS 1) June 22, 2013, off the coast of Malaysia '
                'during Cooperation Afloat Readiness and Training (CARAT) '
                'Malaysia 2013.',
 'name': 'U.S._Navy_Engineman_1st_Class_Don_Barron,_right,_blows_out_the_candles_on_his_birthday_cake_given_to_him_as_a_surprise_by_the_engineering_department_aboard_the_littoral_combat_ship_USS_Freedom_(LCS_1)_June_22_130622-N-JN664-015',
 'page': 'https://commons.wikimedia.org/wiki/File:U.S._Navy_Engineman_1st_Class_Don_Barron%2C_right%2C_blows_out_the_candles_on_his_birthday_cake_given_to_him_as_a_surprise_by_the_engineering_department_aboard_the_littoral_combat_ship_USS_Freedom_(LCS_1)_June_22_130622-N-JN664-015.jpg',
 'src': 'https://upload.wikimedia.o

                'and Law & Justice, Shri Ravi Shankar Prasad presented the '
                'appreciation certificates, at the inauguration of the '
                'National Meet on Grassroot Informatics - Weaving a Digital '
                'India, organised by the National Informatics Centre (NIC), in '
                'New Delhi on January 19, 2017.',
 'name': 'Ravi_Shankar_Prasad_presented_the_appreciation_certificates,_at_the_inauguration_of_the_National_Meet_on_Grassroot_Informatics_-_Weaving_a_Digital_India,_organised_by_the_National_Informatics_Centre_(NIC),_in_New_Delhi_(2)',
 'page': 'https://commons.wikimedia.org/wiki/File:Ravi_Shankar_Prasad_presented_the_appreciation_certificates%2C_at_the_inauguration_of_the_National_Meet_on_Grassroot_Informatics_-_Weaving_a_Digital_India%2C_organised_by_the_National_Informatics_Centre_(NIC)%2C_in_New_Delhi_(2).jpg',
 'src': 'https://upload.wikimedia.org/wikipedia/commons/thumb/6/65/Ravi_Shankar_Prasad_presented_the_appreciation_certifi

10012461
{'description': 'Andrew Aydin, a telecommunications and technology policy aide '
                'to civil rights leader and U.S. Congressman John Lewis, at a '
                'November 7, 2013 book signing for March Book One, the first '
                "volume of Lewis' three-volume graphic novel autobiography, "
                'which Aydin co-wrote, at Midtown Comics Downtown in '
                'Manhattan.',
 'name': '11.7.13AndrewAydinByLuigiNovi2',
 'page': 'https://commons.wikimedia.org/wiki/File:11.7.13AndrewAydinByLuigiNovi2.jpg',
 'src': 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/ee/11.7.13AndrewAydinByLuigiNovi2.jpg/489px-11.7.13AndrewAydinByLuigiNovi2.jpg'}
10012670
{'description': 'The Old South Meeting House in downtown Boston, surrounded by '
                'a bevy of more modern buildings: the Old South Building '
                '(1903; directly to the left of the church building); One '
                'Federal Street (1975; to the left of th

                'public domain, and again without any condition whatsoever).',
 'name': '04671jfChapel_Santuario_de_San_Nicolas_Memorial_Park,_Bulakan,_Bulacanfvf_03',
 'page': 'https://commons.wikimedia.org/wiki/File:04671jfChapel_Santuario_de_San_Nicolas_Memorial_Park%2C_Bulakan%2C_Bulacanfvf_03.jpg',
 'src': 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/04671jfChapel_Santuario_de_San_Nicolas_Memorial_Park%2C_Bulakan%2C_Bulacanfvf_03.jpg/800px-04671jfChapel_Santuario_de_San_Nicolas_Memorial_Park%2C_Bulakan%2C_Bulacanfvf_03.jpg'}
10186093
{'description': ' Top left:The central street of Dubrovnik, theStradun, in '
                'ruins during theSiege of DubrovnikTop right: Vukovar water '
                'tankMiddle left: A Serbian T-55 main battle tank destroyed by '
                'the Croatian Army on the road to DrnišBottom left: Vukovar '
                'memorial cemeteryBottom right: Soldiers of the Croatian Army '
                'getting ready to destroy a Ser

{'description': 'Bornean orangutan (Pongo pygmaeus), Tanjung Putting National '
                'Park - like other great apes, such as chimpanzees, gorillas '
                'and humans, a female will usually have a baby (or two) with '
                'her, but males like to be alone - taken during a photo trip '
                'to Indonesia in 2018 - by Thomas Fuhrmann - see more pictures '
                'on www.snowmanstudios.de',
 'name': 'Bornean_orangutan_(Pongo_pygmaeus),_Tanjung_Putting_National_Park_15',
 'page': 'https://commons.wikimedia.org/wiki/File:Bornean_orangutan_(Pongo_pygmaeus)%2C_Tanjung_Putting_National_Park_15.jpg',
 'src': 'https://upload.wikimedia.org/wikipedia/commons/thumb/7/7c/Bornean_orangutan_(Pongo_pygmaeus)%2C_Tanjung_Putting_National_Park_15.jpg/1200px-Bornean_orangutan_(Pongo_pygmaeus)%2C_Tanjung_Putting_National_Park_15.jpg'}
10079812
{'description': ' Borneanorangutan(Pongo pygmaeus), Tanjung Putting National '
                'Park - like other g

                'Luis, Pampanga National High School Santa Cruz Poblacion-San '
                'Sebastian, San Luis, Pampanga Farm to Market Road Paddy '
                'fields of San Luis, Pampanga in Barangays Santa Lucia '
                '152\'40"N 12046\'32"E San Sebastian 152\'52"N',
 'name': '09802jfSanto_Tomas_San_Luis_High_School_Pampanga_Landmarksfvf_39',
 'page': 'https://commons.wikimedia.org/wiki/File:09802jfSanto_Tomas_San_Luis_High_School_Pampanga_Landmarksfvf_39.jpg',
 'src': 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/09802jfSanto_Tomas_San_Luis_High_School_Pampanga_Landmarksfvf_39.jpg/800px-09802jfSanto_Tomas_San_Luis_High_School_Pampanga_Landmarksfvf_39.jpg'}
10044811
{'description': 'Near Sunset photos - Welcome road-signs of Barangay '
                'Mabalas-balas, San Rafael, Bulacan beside Barangay Cruz na '
                'Daan, San Rafael, Bulacan - Centro, Proper, with the '
                'landmarks - of the Barangay Hall beside the Chape

 'page': 'https://commons.wikimedia.org/wiki/File:09893jfLa_Torre_Dinarayat_Rice_Fields_Highways_Talavera_Nueva_Ecijafvf_14.JPG',
 'src': 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/0e/09893jfLa_Torre_Dinarayat_Rice_Fields_Highways_Talavera_Nueva_Ecijafvf_14.JPG/1280px-09893jfLa_Torre_Dinarayat_Rice_Fields_Highways_Talavera_Nueva_Ecijafvf_14.JPG'}
10271625
{'description': 'MacArthur Highway (San Simon-Apalit, Pampanga section) '
                'Barangay San Agustin  1458\'51"N   12045\'10"E San Jose, San '
                'Simon, Pampanga MacArthur Highway (Note: Judge Florentino '
                'Floro, the owner, to repeat, Donor Florentino Floro of all '
                'these photos hereby donate gratuitously, freely and '
                'unconditionally all these photos to and for Wikimedia '
                'Commons, exclusively, for public use of the public domain, '
                'and again without any condition whatsoever).',
 'name': '8554Simon,_Pampanga_Road

In [49]:
des = img_meta_upd['10027634']['description']
print(des)
D = nlp(des)
for s in D.sents:
    print(s)

8990 Urban Decca Homes Vitas Pumping Station (Estero de Vitas, MMDA, Tondo, Manila) Honorio Lopez Bridge (Velasquez, Infanta & Vitas Streets, Tondo, Manila) Manila Infanta Street -Earnshaw Street Honorio Lopez Bridge Honorio Boulevard, North Bay, Balut, Tondo, Manila Sta. 0+172.68 Nepomuceno Street and Velasquez Street 1437'17"N 12057'54"E Velasquez Street 1437'9"N 12057'56"E  in List of barangays of Metro Manila, Legislative districts of Manila Barangays 94, 95, 96, Zone 8, District II, to Barangays 161, Zone 14, 173 and 179, District II, Gagalangin, Barangays 91, 92, 93, 98, 99, 100, 101, 102, 103, 104, Zone 8, 105, 106, 107, 108, 109, Zone 9, District I, Vitas, Balut & Capulong, Tondo, Manila, City of Manila, surrounded by List of rivers and estuaries in Metro Manila (Note: Judge Florentino Floro, the owner, to repeat, Donor Florentino Floro of all these photos hereby donate gratuitously, freely and unconditionally all these photos to and for Wikimedia Commons, exclusively, for publ

### Check description lengths for x_distractors

In [96]:
txt_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0823_clean_te.json", "r"))
print(Counter([txt_dataset[k]['split'] for k in txt_dataset]))
print(len(set([txt_dataset[k]['Guid'] for k in txt_dataset])))

Counter({'train': 17812, 'test': 4076, 'val': 2455})
24343


In [99]:
for k in txt_dataset:
    for im in txt_dataset[k]['img_negFacts']:
        if len(im['caption'])<5:
            print(im['caption']) 

Ra. 
Ra. 
War
Red
Hu. 
40. 
40. 
War
2013
M198
 Ion
War
 Ch
Ægir
Area
Kick
Ali
Red
 Ion
Area
Si–X
Area
 Ion
 Ion
 Ion
Area
Type
F3. 
F3. 
Si–X
Type
War
War
Ali
 Ion
Type
Si–X
 Ion
War
2
Kick
